***Créer un modèle knn***

**I-Importer les dépendencies**

In [69]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

**II-Importer les données en appelant la source**

In [76]:
df=pd.read_csv("/home/matthieumouillon/code/arundelljr_MoneyBaller/MoneyBaller/raw_data/FC26_20250921.csv")


/tmp/ipykernel_23532/3425015327.py:1: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/home/matthieumouillon/code/arundelljr_MoneyBaller/MoneyBaller/raw_data/FC26_20250921.csv")


In [74]:
#travail avec des données non numériques
# OHE player primary positions

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[['club_position']])
df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['club_position']])


In [79]:
#test avec une colonne à retravailler
club_position_review=df['player_positions'].str.partition(sep=',', expand=True)[0]
club_position_review.head(5)

0    CAM
1     CM
2    CDM
3     RB
4     CM
Name: 0, dtype: object

In [82]:
#il manque une étape : la trsnformation en string s'effectue corectement mais comment appliquer le fit and transform ?

**III-Créer des listes de données numériques et catégorielles**

In [5]:
categorical_columns=['player_id','long_name','player_positions','player_face_url','player_url','short_name','fifa_update_date']
categorical_columns

['player_id',
 'long_name',
 'player_positions',
 'player_face_url',
 'player_url',
 'short_name',
 'fifa_update_date']

In [84]:
numeric_columns=['potential','value_eur']
Xt=numeric_columns

In [85]:
# Test using all features and instanciating the model as 'knn_all'
from sklearn.neighbors import NearestNeighbors
knn_all = NearestNeighbors(
    n_neighbors=6,        # 1 self + 5 similar players
    metric='cosine'       # best for similarity in high dimensions
)
knn_all.fit(X_all_proj)

NearestNeighbors(metric='cosine', n_neighbors=6)

**IV - Choisir et appliquer une standardisation avec input et scale**

In [62]:
#mémo : valeurs numériques choix des colonnes numériques et catégorielles, mais ne pas oublier de les raccrocher au dataframe général

In [65]:
mm_scaler = MinMaxScaler()
df[numeric_columns] = mm_scaler.fit_transform(player_features_df[numeric_columns])
X_all_proj=df[numeric_columns] 
X_all_proj

,potential,value_eur
0,0.978261,1.000000
1,0.891304,0.690544
2,0.869565,0.492837
3,0.891304,0.636103
4,0.826087,0.455587
...,...,...
18400,0.108696,0.000344
18401,0.152174,0.000458
18402,0.217391,0.000516
18403,0.260870,0.000630


In [59]:
# Function only producing similar alternatives based on selected features
def find_similar_players_all(player_id):
    # Step 1: Find correct row index of the player
    player_index = df.index[df['player_id'] == player_id][0]
    # Step 2: Get nearest neighbors with method kneighbhbor
    distances, indices = knn_all.kneighbors([X_all_proj.iloc[player_index]])
